# Prepare Annotation Tables


In [1]:
import pandas as pd
import math
import Bio
from Bio import Seq
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.Data.CodonTable
table = Bio.Data.CodonTable.ambiguous_dna_by_id[1]
import glob
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO

In [2]:
dicSites={}
for i in glob.iglob(r'*/*coordinates*'):
    print(i)
    gene=i.split('/')[0]
    dicSites[gene]=pd.read_csv(i,header=0, index_col='NC_045512',sep='\t')

RdRp/RdRp_coordinates_conversion.tab
3CLpro/3CLpro_coordinates_conversion.tab
N/N_coordinates_conversion.tab
S/S_coordinates.tab
ORF1ab/ORF1ab_coordinates.tab
M/M_coordinates_conversion.tab
ORF3/ORF3_coordinates.tab
helicase/helicase_coordinates.tab


## Build Tables

In [738]:
!ls

3CLpro                           accession_data.tsv
M                                data.numbers
Merge_coordinate_tables.ipynb    epitopes table.numbers
N                                helicase
NT transition 0.05 AF filter.pdf raw_tables
ORF1ab                           residue_annotation
ORF3                             selection_var.csv
Prepare_annotation_tables.ipynb  selection_var.numbers
README.md                        selection_var_21052020.csv
RdRp                             selection_var_31052020.csv
S                                selection_variants_2.numbers
SARS signature for Covid-19.pdf  selection_variants_21052020.csv
Selection_Variants.tsv           sequence.gb
Untitled.ipynb


In [780]:
recs = [rec for rec in SeqIO.parse("sequence.gb", "genbank")]
genome_seq=recs[0].seq

In [781]:
tmpseq=genome_seq[13467-1:21555+1]
protseq=""
for i in range(1,len(tmpseq),3) :
    residue=Bio.Seq._translate_str(tmpseq[i:i+3], table)
    protseq=protseq+residue
print(protseq)

RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYYCKSHKP

/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/Bio/Seq.py:175: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  warnings.warn(
/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/Bio/Seq.py:2855: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [782]:
(13467-13441)

26

In [783]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13468, 'end': 21555, 'seq':str(genome_seq[13468-1:21555+1]) , 'protseq':protseq , 'short':'ORF1b'}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" ]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        seq=genome_seq[start-1:end+1]
        protseq=""
        short=product.split(' ')[0]
        if(product=='RNA-dependent RNA polymerase'):
            seq=genome_seq[13441-1:13468]+genome_seq[13468-1:16237+1]
            short="RdRp"
            print(short)
        elif(product=='3C-like proteinase'):
            short="3CLpro"
            print(short)
        for i in range(1,len(seq),3) :
            residue=Bio.Seq._translate_str(seq[i:i+3], table)
            protseq=protseq+residue
        res={'gene':product,'start':start,'end':end, 'seq':str(seq), 'protseq':protseq , 'short':short}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS" ]
    for feat in feats:
        product=feat.qualifiers['gene'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        seq=genome_seq[start-1:end+1]
        protseq=""
        short=product.split(' ')[0]
        for i in range(1,len(seq),3) :
            residue=Bio.Seq._translate_str(seq[i:i+3], table)
            protseq=protseq+residue
        res={'gene':product,'start':start,'end':end, 'seq':str(seq), 'protseq':protseq , 'short':short}
        features_list.append(res)

3CLpro
RdRp
3CLpro


In [784]:
(21555-13468)/3

2695.6666666666665

In [785]:
features_mat=pd.DataFrame(features_list)

In [786]:
features_mat

,gene,start,end,seq,protseq,short
0,ORF1b polyprotein,13468,21555,CGGGTTTGCGGTGTAAGTGCAGCCCGTCTTACACCGTGCGGCACAG...,RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCR...,ORF1b
1,leader protein,265,805,GATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAA...,MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHL...,leader
2,nsp2,805,2719,GGCATACACTCGCTATGTCGATAACAACTTCTGTGGCCCTGATGGC...,AYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKR...,nsp2
3,nsp3,2719,8554,TGCACCAACAAAGGTTACTTTTGGTGATGACACTGTGATAGAAGTG...,APTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVEL...,nsp3
4,nsp4,8554,10054,TAAAATTGTTAATAATTGGTTGAAGCAGTTAATTAAAGTTACACTT...,KIVNNWLKQLIKVTLVFLFVAAIFYLITPVHVMSKHTDFSSEIIGY...,nsp4
5,3C-like proteinase,10054,10972,GAGTGGTTTTAGAAAAATGGCATTCCCATCTGGTAAAGTTGAGGGT...,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...,3CLpro
6,nsp6,10972,11842,AAGTGCAGTGAAAAGAACAATCAAGGGTACACACCACTGGTTGTTA...,SAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFLYENAFLPFA...,nsp6
7,nsp7,11842,12091,GTCTAAAATGTCAGATGTAAAGTGCACATCAGTAGTCTTACTCTCA...,SKMSDVKCTSVVLLSVLQQLRVESSSKLWAQCVQLHNDILLAKDTT...,nsp7
8,nsp8,12091,12685,AGCTATAGCCTCAGAGTTTAGTTCCCTTCCATCATATGCAGCTTTT...,AIASEFSSLPSYAAFATAQEAYEQAVANGDSEVVLKKLKKSLNVAK...,nsp8
9,nsp9,12685,13024,GAATAATGAGCTTAGTCCTGTTGCACTACGACAGATGTCTTGTGCT...,NNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLS...,nsp9


In [787]:
features_mat=features_mat.drop_duplicates().sort_values(by=['start']).reset_index()[['gene','start','end','seq','protseq','short']]

In [788]:
features_mat[features_mat.gene=='RNA-dependent RNA polymerase']

,gene,start,end,seq,protseq,short
13,RNA-dependent RNA polymerase,13441,16236,GTCAGCTGATGCACAATCGTTTTTAAACCGGGTTTGCGGTGTAAGT...,SADAQSFLNRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFA...,RdRp


---

---

In [789]:
dicSize={}
for i,row in features_mat.iterrows():
    feat=row['short']
    size=(row.end+1-row.start-1)/3
    dicSize[feat]=int(size)

In [790]:
i

28

In [791]:
dicSize

{'leader': 180,
 'ORF1ab': 4406,
 'nsp2': 638,
 'nsp3': 1945,
 'nsp4': 500,
 '3CLpro': 306,
 'nsp6': 290,
 'nsp7': 83,
 'nsp8': 198,
 'nsp9': 113,
 'nsp10': 139,
 'nsp11': 13,
 'RdRp': 931,
 'ORF1b': 2695,
 'helicase': 601,
 "3'-to-5'": 527,
 'endoRNAse': 346,
 "2'-O-ribose": 298,
 'S': 1274,
 'ORF3a': 276,
 'E': 76,
 'M': 223,
 'ORF6': 62,
 'ORF7a': 122,
 'ORF7b': 44,
 'ORF8': 122,
 'N': 420,
 'ORF10': 39}

In [792]:
dicSize.keys()

dict_keys(['leader', 'ORF1ab', 'nsp2', 'nsp3', 'nsp4', '3CLpro', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'nsp11', 'RdRp', 'ORF1b', 'helicase', "3'-to-5'", 'endoRNAse', "2'-O-ribose", 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF7b', 'ORF8', 'N', 'ORF10'])

In [793]:
i

28

In [823]:
dicResAnnot={}
for i in dicSize.keys():
    sites=range(1,int((dicSize[i]))+1)
    dicResAnnot[i]=pd.DataFrame(sites,columns =['NC_045512']) 
    dicResAnnot[i]['Annotation']=''
    dicResAnnot[i]['Impact']=''
    dicResAnnot[i]['Reference']=''
    dicResAnnot[i]=dicResAnnot[i].set_index('NC_045512')

In [824]:
for i in dicSize.keys():
    dicResAnnot[i]['AA']=''
    dicResAnnot[i]['Genome']=''
    start=features_mat[(features_mat.short==i) | (features_mat.gene==i)].start.values[0]+1
    for lin,row in  dicResAnnot[i].iterrows():
        dicResAnnot[i].loc[lin,'AA']=features_mat[(features_mat.short==i) | (features_mat.gene==i)].protseq.values[0][lin-1]
        dicResAnnot[i].loc[lin,'Genome']=((lin-1)*3)+1+start

In [825]:
dicResAnnot.keys()

dict_keys(['leader', 'ORF1ab', 'nsp2', 'nsp3', 'nsp4', '3CLpro', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'nsp11', 'RdRp', 'ORF1b', 'helicase', "3'-to-5'", 'endoRNAse', "2'-O-ribose", 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF7b', 'ORF8', 'N', 'ORF10'])

In [826]:
dicResAnnot['helicase'][['AA','Genome','Annotation','Impact','Reference']]

,AA,Genome,Annotation,Impact,Reference
NC_045512,,,,,
1,A,16238,,,
2,V,16241,,,
3,G,16244,,,
4,A,16247,,,
5,C,16250,,,
...,...,...,...,...,...
597,V,18026,,,
598,A,18029,,,
599,T,18032,,,


In [827]:
features_mat[(features_mat.short==i) | (features_mat.gene==i)].start.values[0]

29557

In [828]:
#separate_variants={}

In [829]:
#for i in dicResAnnot.keys():
#    separate_variants[i]= sel_var[(sel_var.ThirdFeat==i) | (sel_var.SecondFeat==i) | (sel_var.MainFeat==i) ]

In [830]:

#for i in separate_variants.keys():
#    dicResAnnot[i]['Variant']='no'
#    if i in list(sel_var.MainFeat):
#        poscol='MainResidue'
#    elif i in list(sel_var.SecondFeat):
#        poscol='SecondResidue'
#    elif i in list(sel_var.ThirdFeat):
#        poscol='ThirdResidue'
#    for j,row in separate_variants[i].iterrows() :
#        dicResAnnot[i].loc[j,'Variant']='yes'
#        dicResAnnot[i].loc[j,'Annotation']=separate_variants[i].loc[j,'Annotation']
#        dicResAnnot[i].loc[j,'Impact']=separate_variants[i].loc[j,'Impact']
#        dicResAnnot[i].loc[j,'Reference']=separate_variants[i].loc[j,'Reference']
#    dicResAnnot[i]=dicResAnnot[i].replace(np.nan,'')

    

In [831]:
#separate_variants['S']

In [832]:
#dicResAnnot['S'].Annotation.unique()

In [833]:
dicResAnnot.keys()

dict_keys(['leader', 'ORF1ab', 'nsp2', 'nsp3', 'nsp4', '3CLpro', 'nsp6', 'nsp7', 'nsp8', 'nsp9', 'nsp10', 'nsp11', 'RdRp', 'ORF1b', 'helicase', "3'-to-5'", 'endoRNAse', "2'-O-ribose", 'S', 'ORF3a', 'E', 'M', 'ORF6', 'ORF7a', 'ORF7b', 'ORF8', 'N', 'ORF10'])

In [834]:
dicResAnnot['S']

,Annotation,Impact,Reference,AA,Genome
NC_045512,,,,,
1,,,,M,21564
2,,,,F,21567
3,,,,V,21570
4,,,,F,21573
5,,,,L,21576
...,...,...,...,...,...
1270,,,,L,25371
1271,,,,H,25374
1272,,,,Y,25377


In [835]:
for i in dicResAnnot.keys():
    dicResAnnot[i].to_csv('residue_annotation/'+i+'_with_res.tsv', sep='\t')

In [820]:
dicResAnnot

{'leader':           Annotation Impact Reference AA Genome
 NC_045512                                      
 1                                      M    267
 2                                      E    270
 3                                      S    273
 4                                      L    276
 5                                      V    279
 ...              ...    ...       ... ..    ...
 175                                    R    789
 176                                    E    792
 177                                    L    795
 178                                    N    798
 179                                    G    801
 
 [179 rows x 5 columns],
 'ORF1ab':           Annotation Impact Reference AA Genome
 NC_045512                                      
 1                                      M    267
 2                                      E    270
 3                                      S    273
 4                                      L    276
 5                    

In [451]:
dicfiles={}
for i in glob.iglob(r'residue_annotation/*'):
    name=i.replace('residue_annotation/','').replace('_annot.tsv','')
    dicfiles[name]=[name,'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/'+i]

In [452]:
dicfiles

{'nsp10': ['nsp10',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp10_annot.tsv'],
 'ORF1b': ['ORF1b',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1b_annot.tsv'],
 'helicase': ['helicase',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/helicase_annot.tsv'],
 'nsp3': ['nsp3',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp3_annot.tsv'],
 'M': ['M',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/M_annot.tsv'],
 'ORF7a': ['ORF7a',
  'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF7a_annot.tsv'],
 'ORF1ab': ['ORF1ab',
  'https://raw.githubusercontent.com

In [453]:
tablefiles=pd.DataFrame.from_dict(dicfiles,orient='index', columns=['gene','url'])

In [454]:
tablefiles

,gene,url
nsp10,nsp10,https://raw.githubusercontent.com/galaxyprojec...
ORF1b,ORF1b,https://raw.githubusercontent.com/galaxyprojec...
helicase,helicase,https://raw.githubusercontent.com/galaxyprojec...
nsp3,nsp3,https://raw.githubusercontent.com/galaxyprojec...
M,M,https://raw.githubusercontent.com/galaxyprojec...
ORF7a,ORF7a,https://raw.githubusercontent.com/galaxyprojec...
ORF1ab,ORF1ab,https://raw.githubusercontent.com/galaxyprojec...
S,S,https://raw.githubusercontent.com/galaxyprojec...
nsp8,nsp8,https://raw.githubusercontent.com/galaxyprojec...
ORF6,ORF6,https://raw.githubusercontent.com/galaxyprojec...


In [457]:
tablefiles.to_csv('residue_annotation/accessurl.tsv', index=None, sep='\t')

In [458]:
!head accessurl.tsv



gene	url
nsp10	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp10_annot.tsv
ORF1b	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1b_annot.tsv
helicase	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/helicase_annot.tsv
nsp3	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/nsp3_annot.tsv
M	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/M_annot.tsv
ORF7a	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF7a_annot.tsv
ORF1ab	https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/residue_annotation/ORF1ab_annot.tsv
S	https://raw.githubusercon